In [1]:
# --- 0. Imports ---
from math import ceil
from helpers.strategy_helper import StrategyHelper

# Distance Based Race Strategy

Below is an example of a code cell.
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [2]:
# --- 1. Core Inputs ---

# Average Lap Time (s)
lap_time_avg = 121.5

# Race Information
race_distance_km = 1000         # Total Race Length (km)
lap_length_km = 5.807           # Lap Length (km)  5.807 km (3.608 mi)

# Fuel Info
tank_capacity = 103.9           # Tank Capacity (litres)
fuel_per_avg = 3.57             # Basic Fuel/Lap  (litres)
fuel_per_max = 3.67
fuel_per_min = 3.46

# Pit Stop Insights
total_pit_time = 62             # iRaceInsight = 01:01.800
service_time = 42               # From Observation (s) iRaceInsight = 00:40.150
tyre_change_time = 21.0         # From Observation (s) iRaceInsight = 15.616

# Calculate the total number of laps.   UI = 173
total_laps = ceil(race_distance_km / lap_length_km)
print(f"Total laps: {total_laps}")

# Pit Lane Loss - not including fuel time (s)
pit_delta = total_pit_time - service_time
print(f"Pit Delta: {pit_delta}s")

# Refueling Rate (l/s)
refuelling_rate = tank_capacity / service_time
print(f"Refuelling Rate: {refuelling_rate:.2f} l/s")

# Free Tyre Stop Litres (Rounded Up)
tyre_change_litres = ceil(tyre_change_time * refuelling_rate)
tyre_change_stint = tyre_change_litres / fuel_per_avg
print(f"Tyre Change Litres: {tyre_change_litres:.2f}l = ~{tyre_change_stint:.2f} laps")

Total laps: 173
Pit Delta: 20s
Refuelling Rate: 2.47 l/s
Tyre Change Litres: 52.00l = ~14.57 laps


In [3]:
# Helpful methods live here
strategy_helper = StrategyHelper(tank_capacity, total_laps)

# Initial Strategy
for f in (fuel_per_min, fuel_per_avg, fuel_per_max):
    stint_laps = strategy_helper.stint_laps(f, tank_capacity)
    s = strategy_helper.stint_plan(f, stint_laps)
    strs = [
        f"fuel/lap: {s['fuel_per_lap']};",
        f"laps/stint: {s['laps_per_stint']};",
        f"stints: {s['stints']};",
        f"stops: {s['stops']};",
        f"last_stint_laps: {s['last_stint_laps']};",
    ]
    print(" ".join(strs))

fuel/lap: 3.46; laps/stint: 30; stints: 6; stops: 5; last_stint_laps: 23;
fuel/lap: 3.57; laps/stint: 29; stints: 6; stops: 5; last_stint_laps: 28;
fuel/lap: 3.67; laps/stint: 28; stints: 7; stops: 6; last_stint_laps: 5;


In [4]:
result = strategy_helper.calculate_stint_plan(
    total_laps=total_laps,
    tank_capacity=tank_capacity,
    fuel_per_lap=fuel_per_max,      # use worst-case fuel burn
    tyre_change_litres=tyre_change_litres
)

print(result)
total_lap_check = 0
for i, laps in enumerate(result["stint_laps"], start=1):
    total_lap_check += laps
    fuel = strategy_helper.fuel_required_per_stint(laps, fuel_per_avg)
    print(f"Stint {i}: {laps} laps → {fuel:.2f} L")
print(f"Total laps: {total_lap_check}")


{'stints': 7, 'stops': 6, 'laps_for_free_tyres': 15, 'stint_laps': [25, 25, 25, 25, 25, 24, 24]}
Stint 1: 25 laps → 89.25 L
Stint 2: 25 laps → 89.25 L
Stint 3: 25 laps → 89.25 L
Stint 4: 25 laps → 89.25 L
Stint 5: 25 laps → 89.25 L
Stint 6: 24 laps → 85.68 L
Stint 7: 24 laps → 85.68 L
Total laps: 173


In [5]:
stint_laps = strategy_helper.stint_laps(fuel_per_max, tank_capacity)
print(f"TL:{total_laps}; SL:{stint_laps}; F/L:{fuel_per_max:.2f}L; TCL:{tyre_change_litres:.2f}L; TC:{tank_capacity}")
result = strategy_helper.build_last_two_even_from_full_stint(
    total_laps=total_laps,
    max_stint_laps=stint_laps,          # from fuel_per_max and tank_capacity
    fuel_per_lap=fuel_per_max,  # worst-case burn
    tyre_change_litres=tyre_change_litres,
    tank_capacity=tank_capacity,
)

print(result)

TL:173; SL:28; F/L:3.67L; TCL:52.00L; TC:103.9
{'stint_laps': [28, 28, 28, 28, 28, 17, 16], 'full_stints_used': 5, 'last_two_total': 33, 'last_two': (17, 16), 'laps_for_free_tyres': 15}


In [6]:
# Leaders Lap Pace
leader_pace = 119
race_time_seconds = total_laps * leader_pace
your_laps = int(race_time_seconds // lap_time_avg)

print("Leader total laps:", total_laps)
print("Race time (s):", race_time_seconds)
print("Your completed laps:", your_laps)

Leader total laps: 173
Race time (s): 20587
Your completed laps: 169


In [7]:
for f in (fuel_per_min, fuel_per_avg, fuel_per_max):
    stint_laps = strategy_helper.stint_laps(f, tank_capacity)
    your_laps_done = strategy_helper.simulate_laps_with_pits(
        total_race_laps=total_laps,
        leader_lap_time=leader_pace,
        leader_pit_loss=total_pit_time,
        leader_stint_laps=stint_laps,
        your_lap_time=lap_time_avg,
        your_pit_loss=total_pit_time,
        your_stint_laps=stint_laps,
    )

    print(f"@ {f} Leader completes {total_laps} vs {your_laps_done} laps for us")

@ 3.46 Leader completes 173 vs 169 laps for us
@ 3.57 Leader completes 173 vs 169 laps for us
@ 3.67 Leader completes 173 vs 169 laps for us


In [8]:
strategy_helper = StrategyHelper(tank_capacity, your_laps_done)

# Initial Strategy
for f in (fuel_per_min, fuel_per_avg, fuel_per_max):
    stint_laps = strategy_helper.stint_laps(f, tank_capacity)
    s = strategy_helper.stint_plan(f, stint_laps)
    strs = [
        f"fuel/lap: {s['fuel_per_lap']};",
        f"laps/stint: {s['laps_per_stint']};",
        f"stints: {s['stints']};",
        f"stops: {s['stops']};",
        f"last_stint_laps: {s['last_stint_laps']};",
    ]
    print(" ".join(strs))

fuel/lap: 3.46; laps/stint: 30; stints: 6; stops: 5; last_stint_laps: 19;
fuel/lap: 3.57; laps/stint: 29; stints: 6; stops: 5; last_stint_laps: 24;
fuel/lap: 3.67; laps/stint: 28; stints: 7; stops: 6; last_stint_laps: 1;


In [10]:
result = strategy_helper.calculate_stint_plan(
    total_laps=your_laps_done,
    tank_capacity=tank_capacity,
    fuel_per_lap=fuel_per_max,      # use worst-case fuel burn
    tyre_change_litres=tyre_change_litres
)

print(result)
total_lap_check = 0
for i, laps in enumerate(result["stint_laps"], start=1):
    total_lap_check += laps
    fuel = strategy_helper.fuel_required_per_stint(laps, fuel_per_max)
    print(f"Stint {i}: {laps} laps → {fuel:.2f} L")
print(f"Total laps: {total_lap_check}")

AssertionError: Illegal stint: 29 laps exceeds tank capacity